In [6]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
google_api_key = os.environ["GOOGLE_API_KEY"]
google_cloud_project = os.environ["GOOGLE_CLOUD_PROJECT"]

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [10]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
docs = TextLoader("be-good.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
)
retriever = vectorstore.as_retriever()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{input}")]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
output = rag_chain.invoke({"input": "What is this article about?"})
print(output["answer"])

This article discusses the idea that making something people want without focusing on making money is essentially a description of a charity. It explores whether this unexpected result is a flaw in the initial principles or a new discovery. The author also touches on explicitly benevolent projects and their accountability.


In [21]:
output = rag_chain.invoke({"input": "What was my previous question about?"})
print(output["answer"])

I'm sorry, I do not have access to previous conversations. Therefore, I don't know what your previous question was about.


# Create a ChatPromptTemplate able to contextualize inputs
- Goal: put the input in context and re-phrase it so we have a contextualized input.
- We will define a new system prompt that instructs the LLM in how to contextualize the input.
Our new ChatPromptTemplate will include:
- The new system prompt.
MessagesPlaceholder, a placeholder used to pass the list of messages included in the chat_history.

In [24]:
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

## Create a Retriever aware of the memory

In [25]:
from langchain.chains import create_history_aware_retriever

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

## Create a Basic Conversational RAG

In [26]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [27]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is this article about?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What was my previous question about?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Your previous question was, "What is this article about?".


## Step 5: Advanced conversational RAG with persistence and session memories

We will store the chat history in a python dictionary. In advanced apps, you will use advanced ways to store chat history.
Associate chat history with user session with the function get_session_history().
Inject chat history into inputs and update it after each interaction using BaseChatMessageHistory and RunnableWithMessageHistory.

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [29]:
conversational_rag_chain.invoke(
    {"input": "What is this article about?"},
    config={
        "configurable": {"session_id": "001"}
    },  # constructs a key "001" in `store`.
)["answer"]

'This article discusses the idea that making something people want without focusing on making money is essentially a description of a charity. It explores whether this unexpected result is a flaw in the initial principles or a new discovery. The author also touches on explicitly benevolent projects and their accountability.'

In [31]:
conversational_rag_chain.invoke(
    {"input": "What was my previous question about?"},
    config={"configurable": {"session_id": "001"}},
)["answer"]

'Your previous question was "What is this article about?".'

In [32]:
for message in store["001"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is this article about?

AI: This article discusses the idea that making something people want without focusing on making money is essentially a description of a charity. It explores whether this unexpected result is a flaw in the initial principles or a new discovery. The author also touches on explicitly benevolent projects and their accountability.

User: What was my previous question about?

AI: Your previous question was "What is this article about?".

